In [44]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from os import listdir
from keras.preprocessing import sequence
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

In [39]:
# load files
from pandas import read_csv
target = read_csv('/home/supradha/Downloads/MovementAAL/dataset/MovementAAL_target.csv', header=0)
group = read_csv('/home/supradha/Downloads/MovementAAL/groups/MovementAAL_DatasetGroup.csv', header=0)
paths = read_csv('/home/supradha/Downloads/MovementAAL/groups/MovementAAL_Paths.csv', header=0)

In [45]:
#load sequence and target into memory
from pandas import read_csv
from os import listdir
sequence = list()
directory = '/home/supradha/Downloads/MovementAAL/dataset'
target = None
for name in listdir(directory):
    file = directory + '/' + name
    if file.endswith('_target.csv'):
        continue
    df = read_csv(file, header=0)
    values = df.values
    sequence.append(values)

In [47]:
target = read_csv('/home/supradha/Downloads/MovementAAL/dataset/MovementAAL_target.csv')
target = target.values[:,1]

In [48]:
sequence[0]

array([[ 0.66667 ,  0.      , -0.85714 , -0.75    ],
       [ 0.57143 ,  0.76    , -0.42857 , -1.      ],
       [ 0.71429 ,  0.88    , -0.52381 , -0.7     ],
       [ 0.57143 ,  0.88    , -0.66667 , -0.75    ],
       [ 0.7619  ,  0.64    , -0.61905 , -0.65    ],
       [ 0.42857 ,  0.64    , -0.71429 , -0.55    ],
       [ 0.71429 ,  0.76    , -0.71429 , -0.85    ],
       [-0.14286 ,  0.72    , -0.85714 , -0.85    ],
       [ 0.52381 ,  0.4     , -0.66667 , -0.75    ],
       [ 0.85714 ,  0.6     , -0.7619  , -0.7     ],
       [ 0.52381 ,  0.64    , -1.      , -0.9     ],
       [ 0.57143 ,  0.36    , -0.95238 , -0.65    ],
       [ 0.71429 ,  0.2     , -0.71429 , -1.      ],
       [ 0.7619  , -0.04    , -0.66667 , -0.8     ],
       [ 0.71429 ,  0.04    , -0.71429 , -0.9     ],
       [ 0.33333 , -0.28    , -0.57143 , -0.6     ],
       [ 0.38095 ,  0.04    , -0.71429 , -0.5     ],
       [ 0.42857 ,  0.04    , -0.19048 , -1.      ],
       [ 0.42857 ,  0.08    , -0.28571 , -0.9 

In [49]:
group = read_csv('/home/supradha/Downloads/MovementAAL/groups/MovementAAL_DatasetGroup.csv')
group = group.values[:,1]

In [50]:
#for finding ideal length
len_sequence = []
for one_seq in sequence:
    len_sequence.append(len(one_seq))
pd.Series(len_sequence).describe()

count    314.000000
mean      42.028662
std       16.185303
min       19.000000
25%       26.000000
50%       41.000000
75%       56.000000
max      129.000000
dtype: float64

In [51]:
#Padding the sequence with the values to max length
to_pad = 129
new_seq = []
for one_seq in sequence:
    len_one_seq = len(one_seq)
    last_val = one_seq[-1]
    n = to_pad - len_one_seq
   
    to_concat = np.repeat(one_seq[-1], n).reshape(4, n).transpose()
    new_one_seq = np.concatenate([one_seq, to_concat])
    new_seq.append(new_one_seq)
final_seq = np.stack(new_seq)

#truncating the sequence to length 60
from keras.preprocessing import sequence
seq_len = 60
final_seq=sequence.pad_sequences(final_seq, maxlen=seq_len, padding='post', dtype='float', truncating='post')

In [52]:
#seperating into training,validation and test datasets
train = [final_seq[i] for i in range(len(groups)) if (groups[i]==2)]
validation = [final_seq[i] for i in range(len(groups)) if groups[i]==1]
test = [final_seq[i] for i in range(len(groups)) if groups[i]==3]

train_target = [targets[i] for i in range(len(groups)) if (groups[i]==2)]
validation_target = [targets[i] for i in range(len(groups)) if groups[i]==1]
test_target = [targets[i] for i in range(len(groups)) if groups[i]==3]

In [53]:
train = np.array(train)
validation = np.array(validation)
test = np.array(test)

train_target = np.array(train_target)
train_target = (train_target+1)/2

validation_target = np.array(validation_target)
validation_target = (validation_target+1)/2

test_target = np.array(test_target)
test_target = (test_target+1)/2

In [55]:
#building and applying LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(seq_len, 4)))
model.add(Dense(1, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 256)               267264    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 267,521
Trainable params: 267,521
Non-trainable params: 0
_________________________________________________________________


In [56]:
#training and checking accuracy on validation data
adam = Adam(lr=0.001)
chk = ModelCheckpoint('best_model.pkl', monitor='val_acc', save_best_only=True, mode='max', verbose=1)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.fit(train, train_target, epochs=200, batch_size=128, callbacks=[chk], validation_data=(validation,validation_target))

Instructions for updating:
Use tf.cast instead.
Train on 106 samples, validate on 104 samples
Epoch 1/200
106/106 [==============================] - 2s 16ms/step - loss: 0.6948 - acc: 0.5000 - val_loss: 0.6896 - val_acc: 0.5673

Epoch 00001: val_acc improved from -inf to 0.56731, saving model to best_model.pkl
Epoch 2/200
106/106 [==============================] - 1s 5ms/step - loss: 0.6909 - acc: 0.5000 - val_loss: 0.6893 - val_acc: 0.5288

Epoch 00002: val_acc did not improve from 0.56731
Epoch 3/200
106/106 [==============================] - 0s 4ms/step - loss: 0.6880 - acc: 0.5377 - val_loss: 0.6913 - val_acc: 0.4904

Epoch 00003: val_acc did not improve from 0.56731
Epoch 4/200
106/106 [==============================] - 0s 4ms/step - loss: 0.6855 - acc: 0.5472 - val_loss: 0.6952 - val_acc: 0.5000

Epoch 00004: val_acc did not improve from 0.56731
Epoch 5/200
106/106 [==============================] - 0s 4ms/step - loss: 0.6831 - acc: 0.5660 - val_loss: 0.6987 - val_acc: 0.5096

Ep


Epoch 00043: val_acc did not improve from 0.59615
Epoch 44/200
106/106 [==============================] - 0s 4ms/step - loss: 0.4092 - acc: 0.8113 - val_loss: 1.0824 - val_acc: 0.5481

Epoch 00044: val_acc did not improve from 0.59615
Epoch 45/200
106/106 [==============================] - 1s 6ms/step - loss: 0.3969 - acc: 0.8302 - val_loss: 1.2524 - val_acc: 0.5481

Epoch 00045: val_acc did not improve from 0.59615
Epoch 46/200
106/106 [==============================] - 0s 4ms/step - loss: 0.3771 - acc: 0.8208 - val_loss: 1.3396 - val_acc: 0.5288

Epoch 00046: val_acc did not improve from 0.59615
Epoch 47/200
106/106 [==============================] - 1s 7ms/step - loss: 0.3905 - acc: 0.8302 - val_loss: 1.3037 - val_acc: 0.5481

Epoch 00047: val_acc did not improve from 0.59615
Epoch 48/200
106/106 [==============================] - 0s 4ms/step - loss: 0.3665 - acc: 0.8396 - val_loss: 1.2184 - val_acc: 0.5288

Epoch 00048: val_acc did not improve from 0.59615
Epoch 49/200
106/106 [==

106/106 [==============================] - 0s 4ms/step - loss: 0.1670 - acc: 0.9151 - val_loss: 1.6924 - val_acc: 0.5288

Epoch 00088: val_acc did not improve from 0.59615
Epoch 89/200
106/106 [==============================] - 0s 4ms/step - loss: 0.1705 - acc: 0.9151 - val_loss: 1.6936 - val_acc: 0.5192

Epoch 00089: val_acc did not improve from 0.59615
Epoch 90/200
106/106 [==============================] - 0s 4ms/step - loss: 0.1584 - acc: 0.9340 - val_loss: 1.7440 - val_acc: 0.5192

Epoch 00090: val_acc did not improve from 0.59615
Epoch 91/200
106/106 [==============================] - 0s 4ms/step - loss: 0.1541 - acc: 0.9434 - val_loss: 1.7881 - val_acc: 0.5385

Epoch 00091: val_acc did not improve from 0.59615
Epoch 92/200
106/106 [==============================] - 0s 4ms/step - loss: 0.1480 - acc: 0.9340 - val_loss: 1.7819 - val_acc: 0.5385

Epoch 00092: val_acc did not improve from 0.59615
Epoch 93/200
106/106 [==============================] - 0s 4ms/step - loss: 0.1366 - acc


Epoch 00132: val_acc did not improve from 0.59615
Epoch 133/200
106/106 [==============================] - 1s 7ms/step - loss: 0.0430 - acc: 0.9906 - val_loss: 2.0858 - val_acc: 0.5577

Epoch 00133: val_acc did not improve from 0.59615
Epoch 134/200
106/106 [==============================] - 1s 5ms/step - loss: 0.0306 - acc: 1.0000 - val_loss: 2.2138 - val_acc: 0.5577

Epoch 00134: val_acc did not improve from 0.59615
Epoch 135/200
106/106 [==============================] - 0s 4ms/step - loss: 0.0795 - acc: 0.9811 - val_loss: 2.2855 - val_acc: 0.5288

Epoch 00135: val_acc did not improve from 0.59615
Epoch 136/200
106/106 [==============================] - 1s 6ms/step - loss: 0.1913 - acc: 0.9528 - val_loss: 2.1899 - val_acc: 0.5385

Epoch 00136: val_acc did not improve from 0.59615
Epoch 137/200
106/106 [==============================] - 0s 4ms/step - loss: 0.1615 - acc: 0.9245 - val_loss: 2.0826 - val_acc: 0.5577

Epoch 00137: val_acc did not improve from 0.59615
Epoch 138/200
106/1

106/106 [==============================] - 1s 7ms/step - loss: 0.0097 - acc: 1.0000 - val_loss: 2.7262 - val_acc: 0.5288

Epoch 00177: val_acc did not improve from 0.59615
Epoch 178/200
106/106 [==============================] - 1s 10ms/step - loss: 0.0090 - acc: 1.0000 - val_loss: 2.7730 - val_acc: 0.5288

Epoch 00178: val_acc did not improve from 0.59615
Epoch 179/200
106/106 [==============================] - 1s 8ms/step - loss: 0.0079 - acc: 1.0000 - val_loss: 2.8094 - val_acc: 0.5385

Epoch 00179: val_acc did not improve from 0.59615
Epoch 180/200
106/106 [==============================] - 1s 7ms/step - loss: 0.0070 - acc: 1.0000 - val_loss: 2.8310 - val_acc: 0.5385

Epoch 00180: val_acc did not improve from 0.59615
Epoch 181/200
106/106 [==============================] - 1s 8ms/step - loss: 0.0065 - acc: 1.0000 - val_loss: 2.8436 - val_acc: 0.5288

Epoch 00181: val_acc did not improve from 0.59615
Epoch 182/200
106/106 [==============================] - 1s 5ms/step - loss: 0.0060

In [57]:
#checking accuracy of model on the test data
model = load_model('best_model.pkl')

from sklearn.metrics import accuracy_score
test_preds = model.predict_classes(test)
accuracy_score(test_target, test_preds)

0.47115384615384615